<a href="https://colab.research.google.com/github/ValentinaBykova/neuron/blob/master/lab5/Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторна робота №5
## студентки КН-31, Бикової Валентини
## Тема: Convolutional Neural Networks and Computer Vision with TensorFlow

## Завдання
#### На основі документу вирішити завдання класифікації зображень їжі для 3 класів з набору даних food101
#### Індекси класів визначити індивідуально за залежностями: i1=0,i2=30,i3=60 (де і1,і2,і3 - індекс класу (починаючи з 0) у відсортованому за алфавітом наборі даних, n - номер за списком групи. 

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import os
import zipfile
import pathlib
import numpy as np
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense

In [2]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip

zip_ref = zipfile.ZipFile("101_food_classes_10_percent.zip", "r")
zip_ref.extractall()
zip_ref.close()

--2022-06-03 09:13:46--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.33.208, 172.253.62.128, 172.253.115.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.33.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip’

101_food_classes_10 100%[===================>]   1.51G   114MB/s    in 14s     

2022-06-03 09:14:00 (115 MB/s) - ‘101_food_classes_10_percent.zip’ saved [1625420029/1625420029]



In [3]:
!ls 101_food_classes_10_percent/train

apple_pie	    eggs_benedict	     onion_rings
baby_back_ribs	    escargots		     oysters
baklava		    falafel		     pad_thai
beef_carpaccio	    filet_mignon	     paella
beef_tartare	    fish_and_chips	     pancakes
beet_salad	    foie_gras		     panna_cotta
beignets	    french_fries	     peking_duck
bibimbap	    french_onion_soup	     pho
bread_pudding	    french_toast	     pizza
breakfast_burrito   fried_calamari	     pork_chop
bruschetta	    fried_rice		     poutine
caesar_salad	    frozen_yogurt	     prime_rib
cannoli		    garlic_bread	     pulled_pork_sandwich
caprese_salad	    gnocchi		     ramen
carrot_cake	    greek_salad		     ravioli
ceviche		    grilled_cheese_sandwich  red_velvet_cake
cheesecake	    grilled_salmon	     risotto
cheese_plate	    guacamole		     samosa
chicken_curry	    gyoza		     sashimi
chicken_quesadilla  hamburger		     scallops
chicken_wings	    hot_and_sour_soup	     seaweed_salad
chocolate_cake	    hot_dog		     shrimp_and_grits
chocolate_mousse    huevo

In [4]:
train_dir = "101_food_classes_10_percent/train/"
test_dir = "101_food_classes_10_percent/test/"

In [5]:
data_dir = pathlib.Path(train_dir)
all_class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))

indexes = [0, 30, 60]

class_names = np.array([])

for i, name in enumerate(all_class_names):
  if i in indexes:
    print(i, name)
    class_names = np.append(class_names, name)

0 apple_pie
30 deviled_eggs
60 lobster_bisque


In [6]:
!mkdir beeftartare_eggsbenedict_misosoup

!mkdir beeftartare_eggsbenedict_misosoup/train
!mkdir beeftartare_eggsbenedict_misosoup/test

In [7]:
sources = [
  'beef_tartare',
  'eggs_benedict',
  'miso_soup'
]

for source in sources:
  source_dir = train_dir + source
  destination_dir = 'beeftartare_eggsbenedict_misosoup/train/' + source
  shutil.copytree(source_dir, destination_dir)

for source in sources:
  source_dir = test_dir + source
  destination_dir = 'beeftartare_eggsbenedict_misosoup/test/' + source
  shutil.copytree(source_dir, destination_dir)

In [8]:
train_dir = "beeftartare_eggsbenedict_misosoup/train/"
test_dir = "beeftartare_eggsbenedict_misosoup/test/"

In [9]:
data_dir = pathlib.Path(train_dir)
class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))
print(class_names)

['beef_tartare' 'eggs_benedict' 'miso_soup']


In [10]:
train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

train_data = train_datagen.flow_from_directory(test_dir,
                                               target_size=(224, 224),
                                               batch_size=32,
                                               class_mode='categorical')

test_data = train_datagen.flow_from_directory(train_dir,
                                              target_size=(224, 224),
                                              batch_size=32,
                                              class_mode='categorical')

Found 750 images belonging to 3 classes.
Found 225 images belonging to 3 classes.


In [11]:
train_datagen_augmented = ImageDataGenerator(rescale=1/255.,
                                             rotation_range=20, 
                                             width_shift_range=0.2,
                                             height_shift_range=0.2,
                                             zoom_range=0.2,
                                             horizontal_flip=True)

train_data_augmented = train_datagen_augmented.flow_from_directory(test_dir,
                                                                  target_size=(224, 224),
                                                                  batch_size=32,
                                                                  class_mode='categorical')

Found 750 images belonging to 3 classes.


In [13]:
cnn_model = Sequential([
  Conv2D(32, 3, activation='relu', input_shape=(224, 224, 3)),
  MaxPool2D(),
  Dense(128, activation='relu'),
  Conv2D(60, 3, activation='relu'),
  MaxPool2D(),
  Flatten(),
  Dense(3, activation='softmax')
])

cnn_model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

result_cnn_model = cnn_model.fit(train_data_augmented,
                          epochs=50,
                          steps_per_epoch=len(train_data_augmented),
                          validation_data=test_data,
                          validation_steps=len(test_data))

Epoch 1/50
24/24 [==============================] - 161s 7s/step - loss: 1.3548 - accuracy: 0.3373 - val_loss: 1.0488 - val_accuracy: 0.3333
Epoch 2/50
24/24 [==============================] - 161s 7s/step - loss: 1.0686 - accuracy: 0.3947 - val_loss: 1.0477 - val_accuracy: 0.5067
Epoch 3/50
24/24 [==============================] - 154s 6s/step - loss: 1.0259 - accuracy: 0.4533 - val_loss: 0.8939 - val_accuracy: 0.5378
Epoch 4/50
24/24 [==============================] - 155s 6s/step - loss: 0.9346 - accuracy: 0.5173 - val_loss: 0.9215 - val_accuracy: 0.5867
Epoch 5/50
24/24 [==============================] - 154s 6s/step - loss: 1.0082 - accuracy: 0.4787 - val_loss: 0.8901 - val_accuracy: 0.5289
Epoch 6/50
24/24 [==============================] - 155s 6s/step - loss: 0.9143 - accuracy: 0.5147 - val_loss: 0.8389 - val_accuracy: 0.6178
Epoch 7/50
24/24 [==============================] - 156s 6s/step - loss: 0.8954 - accuracy: 0.5693 - val_loss: 0.8005 - val_accuracy: 0.6622
Epoch 8/50
24

## Висновок
#### Під час даної лабораторної роботи я розібрала багатокласову класифікацію за допомогою нейромережі Convolutional Neural Networks. А саме, створила нейромережу для розвʼязання задачі класифікації зображень їжі.